In [235]:
from pyspark.sql import SparkSession
from pyspark.sql.types import LongType
from pyspark.sql import functions as F
from pyspark.sql.functions import date_format
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.functions import isnan, when, count, col
import pandas as pd
from urllib.request import urlretrieve
from zipfile import ZipFile
# Import libraries 

from pyspark.sql import SparkSession, functions as F
from pyspark.sql.functions import isnan, when, count, col
import pandas as pd
import geopandas as gpd
import folium
from chart_studio import plotly as py
import plotly.graph_objs as go
import statsmodels.api as sm
from statsmodels.formula.api import ols
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.lines import Line2D

from pyspark.sql.functions import col,isnan, when, count
import os

In [1]:

#--------------------------------------------------------------------------------------------
# Create a spark session
spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "10g")
    .getOrCreate()
)

22/09/11 20:34:51 WARN Utils: Your hostname, MacBook-Air-3.local resolves to a loopback address: 127.0.0.1; using 192.168.0.66 instead (on interface en0)
22/09/11 20:34:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/11 20:34:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [48]:
exchange_rates = spark.read.csv("../data/exchange_rates.csv")
inflation = spark.read.csv("../data/inflation.csv")
merch_fees = spark.read.csv("../data/merch_fees.csv")
covid = spark.read.csv("../data/covid.csv", header=True)

In [49]:
covid.show(5)

+----------+--------------------+------------+---------+------+-----+---------+---------+----+---+----+--------+
|      date|               state|state_abbrev|confirmed|deaths|tests|positives|recovered|hosp|icu|vent|vaccines|
+----------+--------------------+------------+---------+------+-----+---------+---------+----+---+----+--------+
|2020-01-25|Australian Capita...|         ACT|        0|     0|    0|        0|        0|   0|  0|   0|       0|
|2020-01-25|     New South Wales|         NSW|        3|     0|    0|        3|        0|   0|  0|   0|       0|
|2020-01-25|  Northern Territory|          NT|        0|     0|    0|        0|        0|   0|  0|   0|       0|
|2020-01-25|          Queensland|         QLD|        0|     0|    0|        0|        0|   0|  0|   0|       0|
|2020-01-25|     South Australia|          SA|        0|     0|    0|        0|        0|   0|  0|   0|       0|
+----------+--------------------+------------+---------+------+-----+---------+---------+----+--

In [11]:
data = spark.read.parquet("../data/tables/full_join.parquet")

In [12]:
data.show(5)

+-------------+------------+--------------------+---------+--------------+---------------+--------------------+-----+--------+------+------------------+------------------+--------------------+--------------+-------+-----------+
|merchant_name|merchant_abn|          categories|take_rate|revenue_levels|           name|             address|state|postcode|gender|trans_merchant_abn|      dollar_value|            order_id|order_datetime|user_id|consumer_id|
+-------------+------------+--------------------+---------+--------------+---------------+--------------------+-----+--------+------+------------------+------------------+--------------------+--------------+-------+-----------+
|Felis Limited| 10023283211|furniture, home f...|     0.18|             e|      Todd Long|883 Patty Mountai...|  NSW|    2021|  Male|       10023283211| 212.8573179553598|2430fff1-36c1-483...|    2021-05-02|   3471|        551|
|Felis Limited| 10023283211|furniture, home f...|     0.18|             e|  Adrian Mille

In [13]:
data.createOrReplaceTempView("temp")

a = spark.sql(""" 

SELECT DISTINCT(merchant_name)
FROM temp

""")

a.show()

+--------------------+
|       merchant_name|
+--------------------+
|      Amet Nulla Ltd|
|Volutpat Nulla In...|
|   Pede Praesent Ltd|
|Etiam Bibendum In...|
|Magna Ut Incorpor...|
|Faucibus Leo In C...|
|Varius Et Associates|
|Tellus Imperdiet ...|
|Molestie Tellus F...|
|Class Aptent Taci...|
|Placerat Orci Ass...|
|Vitae Purus Gravi...|
|    Dictum Magna Ltd|
|Sed Leo Cras Inco...|
|    Dui Incorporated|
|        Et Pede Inc.|
| Sagittis Associates|
|Ante Nunc Mauris ...|
|Imperdiet Ullamco...|
|     Ut Molestie LLP|
+--------------------+
only showing top 20 rows



In [219]:
SA2_CENSUS_URL = "https://www.abs.gov.au/census/find-census-data/datapacks/download/2021_GCP_SA2_for_AUS_short-header.zip"

output_csv = "../data/SA2_census/census.zip"

urlretrieve(SA2_CENSUS_URL, output_csv) 

with ZipFile(output_csv, 'r') as zip:
    # extracting all the files
    zip.extractall(path = "../data/SA2_census/")

census = pd.read_csv("../data/SA2_census/2021 Census GCP Statistical Area 2 for AUS/2021Census_G01_AUST_SA2.csv")

census.dropna()
census.to_csv("../data/curated/SA2_census.csv")

('../data/SA2_census/census.zip', <http.client.HTTPMessage at 0x7fa3ec534d60>)

In [245]:
census

,SA2_CODE_2021,Tot_P_M,Tot_P_F,Tot_P_P,Age_0_4_yr_M,Age_0_4_yr_F,Age_0_4_yr_P,Age_5_14_yr_M,Age_5_14_yr_F,Age_5_14_yr_P,...,High_yr_schl_comp_Yr_8_belw_P,High_yr_schl_comp_D_n_g_sch_M,High_yr_schl_comp_D_n_g_sch_F,High_yr_schl_comp_D_n_g_sch_P,Count_psns_occ_priv_dwgs_M,Count_psns_occ_priv_dwgs_F,Count_psns_occ_priv_dwgs_P,Count_Persons_other_dwgs_M,Count_Persons_other_dwgs_F,Count_Persons_other_dwgs_P
0,101021007,2234,2117,4343,111,98,207,278,210,487,...,103,5,10,14,1979,1902,3878,288,251,535
1,101021008,4264,4255,8517,276,252,526,595,495,1088,...,366,21,27,43,3989,3961,7950,240,220,460
2,101021009,5704,5638,11342,318,349,669,551,516,1067,...,410,35,52,83,5264,5143,10410,476,467,943
3,101021010,2647,2440,5085,163,164,319,253,216,469,...,116,10,12,27,2462,2344,4800,188,111,302
4,101021012,6331,6413,12744,416,421,835,984,896,1878,...,258,11,0,18,6079,6188,12260,175,187,358
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2467,901021002,302,292,593,23,15,37,49,39,88,...,19,38,47,86,247,238,491,113,106,222
2468,901031003,149,158,310,7,14,14,18,29,46,...,15,0,0,0,125,145,267,121,53,178
2469,901041004,1060,1130,2188,59,41,101,139,114,247,...,27,3,0,4,878,962,1842,220,221,443
2470,997979799,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


,SA2_CODE_2021,Tot_P_M,Tot_P_F,Tot_P_P,Age_0_4_yr_M,Age_0_4_yr_F,Age_0_4_yr_P,Age_5_14_yr_M,Age_5_14_yr_F,Age_5_14_yr_P,...,High_yr_schl_comp_Yr_8_belw_P,High_yr_schl_comp_D_n_g_sch_M,High_yr_schl_comp_D_n_g_sch_F,High_yr_schl_comp_D_n_g_sch_P,Count_psns_occ_priv_dwgs_M,Count_psns_occ_priv_dwgs_F,Count_psns_occ_priv_dwgs_P,Count_Persons_other_dwgs_M,Count_Persons_other_dwgs_F,Count_Persons_other_dwgs_P
0,101021007,2234,2117,4343,111,98,207,278,210,487,...,103,5,10,14,1979,1902,3878,288,251,535
1,101021008,4264,4255,8517,276,252,526,595,495,1088,...,366,21,27,43,3989,3961,7950,240,220,460
2,101021009,5704,5638,11342,318,349,669,551,516,1067,...,410,35,52,83,5264,5143,10410,476,467,943
3,101021010,2647,2440,5085,163,164,319,253,216,469,...,116,10,12,27,2462,2344,4800,188,111,302
4,101021012,6331,6413,12744,416,421,835,984,896,1878,...,258,11,0,18,6079,6188,12260,175,187,358
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2467,901021002,302,292,593,23,15,37,49,39,88,...,19,38,47,86,247,238,491,113,106,222
2468,901031003,149,158,310,7,14,14,18,29,46,...,15,0,0,0,125,145,267,121,53,178
2469,901041004,1060,1130,2188,59,41,101,139,114,247,...,27,3,0,4,878,962,1842,220,221,443
2470,997979799,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
